<a href="https://colab.research.google.com/github/chelseama0715/Yuchen-Ma/blob/main/Heston_model(Ordinary_RE).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Option pricing in the (rough) Heston model（fractional Riccati equation）



In [1]:
from scipy.stats import norm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
 

In the rough Heston model, the pair (S,V)of the stock price and its instantaneous variance has the dynamics $\mathrm{d}S_t=S_t\sqrt{V_t}\mathrm{d}W_t$ , $S_0=s_0\in R_+$
$V_t=V_0+\frac{1}{\Gamma(\alpha)}\int_{0}^{t}(t-s)^{\alpha-1}\eta(m-V_s)\mathrm{d}s+\frac{1}{\Gamma(\alpha)}\int_{0}^{t}(t-s)^{\alpha-1}\zeta\sqrt{V_s}\mathrm{d}B_s , V_0\in R_+$
where $\eta,m,\zeta$ are positive real numbers, and the correlation between the two Brownian motion W and B is $\rho \in (-1,1)$. The parameter $\alpha \in (0,2)$ plays an important role in representing the Hurst parameter. 

 First, consider the case where the model is not rough(**$\alpha$ = 1**),
 then the equation will be reduced to $\psi^{'} = \lambda\psi^2+\mu\psi+\nu$

We get the solution: $\psi(t) = \sum a_kt^k$ with $a_0=0,a_1=\nu/\Gamma(2),a_{k+1}=(\lambda a_k^{*2}+\mu a_k)/(k+1)$


# inputs:  
$\lambda - \text{second order coefficient of the equation} \\ 
\mu - \text{first order coefficient of the equation} \\
\nu - \text{the intercept of the equation}\\
t - \text{time}$

# output:

**Solutions of the equation: $\psi(t)$**



In [2]:
# generate random data in matrix[:,400] using monte carlo sampling

lambda_1 = np.random.uniform(0.0001,0.001,400)
mu = np.random.uniform(-3,-1,400)
nu = np.random.uniform(18,20,400)

t = np.random.uniform(0,1/252)


r_max = 150

def return_all_the_coefficients():  # return the list of coefficients up to a_k

  coefficients = np.empty((r_max+1,400))  # initialize the coefficient in the form of matrix

  a0 = 0
  a1 = nu
  coefficients[0,:] = a0
  coefficients[1,:] = a1

  def second_order_coefficients_convol(list_of_coefficients,k): #return a^*2_k given the first k-1 coefficients
    if k==1:
      return 0
    else:
      sum1 = 0
      for l in range(1,k):
        a_l = list_of_coefficients[l,:]
        a_k_l = list_of_coefficients[k-l,:]
        sum1 += a_l*a_k_l
      return sum1

  def second_order_coefficients(list_of_coefficients,n): #return the coefficient a_n given the discrete time convolution equation
    k = n-1
    a_k_star_quadratic = second_order_coefficients_convol(list_of_coefficients,k)
    a_k = list_of_coefficients[k,:]
    a_n = (lambda_1*a_k_star_quadratic + mu*a_k)/(k+1)

    return a_n

  for i in range(2,r_max+1):
    coefficients[i,:] = second_order_coefficients(coefficients,i)

  return coefficients
  
 

  
  

In [3]:
 coeff=return_all_the_coefficients()
 coeff

array([[ 0.00000000e+000,  0.00000000e+000,  0.00000000e+000, ...,
         0.00000000e+000,  0.00000000e+000,  0.00000000e+000],
       [ 1.93904840e+001,  1.95147321e+001,  1.92389295e+001, ...,
         1.83370732e+001,  1.83709715e+001,  1.87942184e+001],
       [-1.61051427e+001, -1.87405195e+001, -2.45237053e+001, ...,
        -2.61792191e+001, -1.16233030e+001, -1.52854649e+001],
       ...,
       [-1.68236458e-074, -1.14352149e-065, -1.39348759e-051, ...,
        -2.06654456e-055, -3.05667667e-101, -1.16827184e-079],
       [ 5.12055510e-075,  3.99604508e-066,  6.06216449e-052, ...,
         8.42334368e-056,  6.09686357e-102,  3.27535106e-080],
       [-1.55852571e-075, -1.39642118e-066, -2.63725624e-052, ...,
        -3.43339893e-056, -1.21608366e-102, -9.18272968e-081]])

In [4]:
len(coeff)


151

In [5]:

coef_df = pd.DataFrame(coeff)
coef_df = coef_df.rename(columns={0:"val"})
coef_df.shape[1]
coef_df

,val,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399
0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,1.939048e+01,1.951473e+01,1.923893e+01,1.864628e+01,1.832754e+01,1.869568e+01,1.860920e+01,1.940246e+01,1.912627e+01,1.939342e+01,1.865342e+01,1.964033e+01,1.898670e+01,1.843963e+01,1.847506e+01,1.809817e+01,1.967126e+01,1.967271e+01,1.900249e+01,1.916277e+01,1.959561e+01,1.866411e+01,1.922248e+01,1.978114e+01,1.999135e+01,1.884075e+01,1.844699e+01,1.839400e+01,1.919170e+01,1.978013e+01,1.857789e+01,1.835413e+01,1.812719e+01,1.805520e+01,1.973472e+01,1.962116e+01,1.976427e+01,1.991597e+01,1.907422e+01,1.896135e+01,...,1.948566e+01,1.953372e+01,1.860578e+01,1.874273e+01,1.895986e+01,1.862735e+01,1.835907e+01,1.828777e+01,1.987379e+01,1.838591e+01,1.963854e+01,1.825749e+01,1.806505e+01,1.869148e+01,1.975093e+01,1.967695e+01,1.817607e+01,1.900819e+01,1.856289e+01,1.979299e+01,1.805855e+01,1.999122e+01,1.856968e+01,1.961521e+01,1.932242e+01,1.883462e+01,1.958655e+01,1.822257e+01,1.859577e+01,1.833870e+01,1.912098e+01,1.808017e+01,1.800233e+01,1.990581e+01,1.874170e+01,1.828034e+01,1.957410e+01,1.833707e+01,1.837097e+01,1.879422e+01
2,-1.610514e+01,-1.874052e+01,-2.452371e+01,-1.583578e+01,-2.618408e+01,-2.420707e+01,-1.111929e+01,-1.973713e+01,-2.418527e+01,-1.655296e+01,-1.938287e+01,-2.551821e+01,-1.274929e+01,-9.948480e+00,-1.437683e+01,-1.786748e+01,-1.337241e+01,-9.893787e+00,-1.274557e+01,-2.600807e+01,-2.938810e+01,-2.156155e+01,-1.945579e+01,-1.562445e+01,-2.235955e+01,-1.002573e+01,-1.204409e+01,-1.800987e+01,-1.050793e+01,-1.478659e+01,-2.645166e+01,-2.466312e+01,-1.321781e+01,-1.676209e+01,-2.836146e+01,-1.985560e+01,-1.397394e+01,-1.307734e+01,-1.927207e+01,-1.849459e+01,...,-1.783640e+01,-1.124053e+01,-1.486250e+01,-2.218159e+01,-2.632035e+01,-2.714339e+01,-1.876052e+01,-9.383211e+00,-2.639428e+01,-2.511766e+01,-2.714509e+01,-2.007106e+01,-2.617775e+01,-9.592923e+00,-1.248277e+01,-1.604509e+01,-1.255296e+01,-2.265597e+01,-1.773757e+01,-1.706074e+01,-1.134681e+01,-2.225784e+01,-2.503263e+01,-1.167433e+01,-2.141790e+01,-1.816244e+01,-2.260005e+01,-2.381408e+01,-1.544966e+01,-1.547244e+01,-1.957081e+01,-2.176333e+01,-2.543186e+01,-1.444822e+01,-1.572751e+01,-2.074904e+01,-9.917953e+00,-2.617922e+01,-1.162330e+01,-1.528546e+01
3,8.996744e+00,1.210032e+01,2.096238e+01,9.061708e+00,2.504410e+01,2.097905e+01,4.474082e+00,1.343619e+01,2.042098e+01,9.530883e+00,1.347079e+01,2.219990e+01,5.764523e+00,3.617990e+00,7.545952e+00,1.186785e+01,6.161973e+00,3.413047e+00,5.790527e+00,2.356628e+01,2.940059e+01,1.665851e+01,1.316563e+01,8.333068e+00,1.672424e+01,3.665982e+00,5.280168e+00,1.177309e+01,3.920187e+00,7.400044e+00,2.519616e+01,2.215997e+01,6.497234e+00,1.044728e+01,2.723454e+01,1.342614e+01,6.649886e+00,5.760512e+00,1.300802e

In [6]:
# return psi_t as the solution of the ORE
psi_t = 0
for k in range(1,r_max+1):
  psi_t += coeff[k,:]*pow(t,k)
   
psi_t



array([0.06628717, 0.06668228, 0.06566914, 0.063739  , 0.06252527,
       0.06381046, 0.06366712, 0.06628572, 0.06528687, 0.06629199,
       0.06372196, 0.06703362, 0.06494219, 0.06309952, 0.0631691 ,
       0.06183614, 0.06728175, 0.0673275 , 0.06499636, 0.0653907 ,
       0.0668351 , 0.06373314, 0.06567201, 0.06763209, 0.0682739 ,
       0.06447377, 0.06310018, 0.06284865, 0.06567127, 0.06763844,
       0.06338041, 0.06263421, 0.06199008, 0.06170178, 0.06732398,
       0.0670341 , 0.06759356, 0.06812413, 0.06516587, 0.064788  ,
       0.06788048, 0.06330862, 0.06700359, 0.06656328, 0.06282807,
       0.06501856, 0.0657793 , 0.06475682, 0.06528685, 0.0642444 ,
       0.06816645, 0.06522587, 0.06576888, 0.06189892, 0.06492516,
       0.06668621, 0.0626747 , 0.06180959, 0.06553757, 0.06734591,
       0.06770651, 0.0622482 , 0.06378289, 0.0645003 , 0.0626476 ,
       0.06262164, 0.06645509, 0.06785001, 0.06724604, 0.06678887,
       0.0679888 , 0.06317389, 0.06805758, 0.06487262, 0.06296

# **Data preprocessing**

In [23]:
coeff_trans = np.transpose(coeff)    #transform the "coeff" into shape(400,151)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(coeff_trans, psi_t, test_size=0.25, train_size=0.75)

In [25]:
# normalize
from sklearn import preprocessing

x_train_scaled = preprocessing.scale(x_train)
y_train_scaled = preprocessing.scale(y_train)
x_test_scaled = preprocessing.scale(x_test)
y_test_scaled = preprocessing.scale(y_test)


# **Create the Neural Network**

In [26]:
import keras
from keras.models import Sequential
from keras.layers import Activation,Dense,Dropout
from keras import backend as K
keras.backend.set_floatx('float64')


#create a neural network in the framework:
# input: coeff    output: psi_t 
# 4 hidden layers   30 neurons on each hidden layer  

model = Sequential()
model.add(Dense(32, activation="elu", input_dim =151))
model.add(Dropout(0.5))
model.add(Dense(32,activation="elu"))
model.add(Dropout(0.25))
model.add(Dense(32,activation="elu"))
model.add(Dropout(0.5))
model.add(Dense(151,activation="softmax"))



# **Fit the neural network**

In [27]:
model.compile(loss= "mse", optimizer = "adam")

model.fit(x_train, y_train, batch_size=32, validation_data = (x_test,
y_test),epochs = 10 , verbose = 'auto', shuffle = True)                                     

Epoch 1/10
10/10 [==============================] - 1s 18ms/step - loss: 0.0037 - val_loss: 0.0034
Epoch 2/10
10/10 [==============================] - 0s 5ms/step - loss: 0.0035 - val_loss: 0.0034
Epoch 3/10
10/10 [==============================] - 0s 5ms/step - loss: 0.0035 - val_loss: 0.0034
Epoch 4/10
10/10 [==============================] - 0s 5ms/step - loss: 0.0034 - val_loss: 0.0034
Epoch 5/10
10/10 [==============================] - 0s 5ms/step - loss: 0.0034 - val_loss: 0.0034
Epoch 6/10
10/10 [==============================] - 0s 5ms/step - loss: 0.0034 - val_loss: 0.0034
Epoch 7/10
10/10 [==============================] - 0s 6ms/step - loss: 0.0034 - val_loss: 0.0034
Epoch 8/10
10/10 [==============================] - 0s 8ms/step - loss: 0.0034 - val_loss: 0.0034
Epoch 9/10
10/10 [==============================] - 0s 7ms/step - loss: 0.0034 - val_loss: 0.0034
Epoch 10/10
10/10 [==============================] - 0s 7ms/step - loss: 0.0034 - val_loss: 0.0034


In [28]:
cost = model.evaluate(x_test_scaled, y_test_scaled, batch_size=40)
print('test cost:', cost)

3/3 [==============================] - 0s 3ms/step - loss: 1.0001
test cost: 1.0000518560409546
